# 제1회 KRX 금융 빅데이터 활용 아이디어 경진대회

## 주제 : 개인투자자의 ktop30 감정지표

- __chapter 1. 정형데이터__
- __chapter 2. 비정형 데이터__
    - 비정형 데이터 크롤링
        - 유튜브댓글 api
        - 네이버금융 종목토론방 댓글 크롤링
    - 비정형 데이터 전처리
        - 댓글 전처리
        - 댓글 토크나이징
        - 댓글 레이블링
        - 학습데이터 생성 (댓글 도미넌스 기반 랜덤추출)
        - 날짜별 데이터 정리
    
- __chapter 3. BERT 모델링__
    - 모델 학습
    - 모델 예측

- __chapter 4. 시각화 및 서비스__

- __chapter 5. Reference__
    - [한국거래소](http://www.krx.co.kr/main/main.jsp)
    - [네이버 금융](https://finance.naver.com/)
    - [CNN Fear & Greed Index](https://edition.cnn.com/markets/fear-and-greed) -> 정형지표만 이용한듯 해요
    - [Crypto Fear & Greed Index](https://alternative.me/crypto/fear-and-greed-index/)
    - 단어집 위치


---

## Abstract
    금융시장은 투자자의 시장인식에 따라 영향이 있을것이라 생각했습니다. 국내에서는 심리를 대변하는 지수로 정형지표의 계산으로 산출된 VIX지수를 지표로 이용합니다. 그러나 Crypto Fear & Greed Index 에서 비정형 데이터로 트위터 데이터를 산출식에 포함시키는 시도가 있었습니다. (가이드글)

    # 개인투자자의 시장인식이 댓글형태로 나타나는 네이버 종목토론실 댓글을 이용합니다. # 개인투자자의 시장인식을 수치화시켜 하나의 지표로 만들었습니다. # KODEX30 으로 프로토타입을 작성했습니다.

    인덱스가 FEAR(0)에 가까울수록 투자자가 느끼는 시장인식은 가격이 하락하고 있음을 대변합니다. 인덱스 활용방법으로는 매수시점을 정하는데 도움을 받을 수 있습니다. 반대의 경우 가격이 상승하는것을 대변하고, 매도시점을 정하는데 도움을 받을 수 있습니다.


- 사용데이터소개 (비정형/정형)
- 인덱스의 기대효과

---

## chapter 1. 정형 데이터

## chapter 2. 비정형 데이터

### 2.1 비정형 데이터 크롤링

### 1.1 유튜브댓글 api

- 유튜브 채널 '삼프로tv'에서 2020년 06월부터 2022년 06월 사이에 업로드된 영상의 댓글을 크롤링하였습니다.
- '영상 제목', '영상 업로드 날짜', '영상 아이디', '댓글 내용', '댓글 작성자', '댓글 작성 날짜', '좋아요 수'에 대한 정보를 가져왔습니다.
- 국내 시장에 대한 반응을 확인하기 위해 해외 시장에 관련된 영상이나 경제와 관련없는 영상에 달린 댓글들은 제외하였습니다.

In [ ]:
import pandas as pd
import datetime
import scrapetube
from googleapiclient.discovery import build

import warnings
warnings.filterwarnings('ignore')

In [ ]:
api_key = 'api_key'
youtube = build('youtube', 'v3', developerKey=api_key)

search_response = youtube.search().list(
    q = '삼프로tv',
    order = 'relevance',
    part = 'snippet',
    maxResults = 10,
    ).execute()

channel_id = search_response['items'][0]['snippet']['channelId']

videos = scrapetube.get_channel(channel_id)

video_ids = []

for video in videos:
    video_ids.append(video['videoId'])

for i in range(800):
    start = i * 50
    end = (i + 1) * 50
    video_request = youtube.videos().list(
        part="snippet",
        id=','.join(video_ids[start:end]))
    
    video_response = video_request.execute()
    
    for item in video_response['items']:
        title = item['snippet']['title']
        if ('글로벌 이슈체크' in title) or ('글로벌 마켓브리핑' in title) or ('직장인 vlog' in title):
            continue
        video_infos.append([item['snippet']['title'], item['snippet']['publishedAt'], item['id']])

df = pd.DataFrame(video_infos, columns=['title', 'video_date', 'id'])

df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
df = df.set_index('date')

df_22 = df[(pd.DatetimeIndex(df.index).year == 2022) & (pd.DatetimeIndex(df.index).month <= 6)]
df_21 = df[(pd.DatetimeIndex(df.index).year == 2021)]
df_20 = df[(pd.DatetimeIndex(df.index).year == 2020) & (pd.DatetimeIndex(df.index).month >= 6)]

comments_22 = []

for video_id in df_22['id']:
    api_obj = build('youtube', 'v3', developerKey=api_key)
    response = api_obj.commentThreads().list(part='snippet', videoId=video_id, maxResults=100).execute()

    while response:
        for item in response['items']:
            comment = item['snippet']['topLevelComment']['snippet']
            comments_22.append([video_id, comment['textDisplay'], comment['authorDisplayName'], comment['publishedAt'], comment['likeCount']])

        if 'nextPageToken' in response:
            response = api_obj.commentThreads().list(part='snippet', videoId=video_id, pageToken=response['nextPageToken'], maxResults=100).execute()
        else:
            break

df1 = pd.DataFrame(comments_22, columns=['id', 'comment', 'author', 'comment_date', 'num_likes'])

comments_21 = []

for video_id in df_21['id']:
    api_obj = build('youtube', 'v3', developerKey=api_key)
    response = api_obj.commentThreads().list(part='snippet', videoId=video_id, maxResults=100).execute()

    while response:
        for item in response['items']:
            comment = item['snippet']['topLevelComment']['snippet']
            comments_21.append([video_id, comment['textDisplay'], comment['authorDisplayName'], comment['publishedAt'], comment['likeCount']])

        if 'nextPageToken' in response:
            response = api_obj.commentThreads().list(part='snippet', videoId=video_id, pageToken=response['nextPageToken'], maxResults=100).execute()
        else:
            break

df2 = pd.DataFrame(comments_21, columns=['id', 'comment', 'author', 'comment_date', 'num_likes'])

comments_20 = []

for video_id in df_21['id']:
    api_obj = build('youtube', 'v3', developerKey=api_key)
    response = api_obj.commentThreads().list(part='snippet', videoId=video_id, maxResults=100).execute()

    while response:
        for item in response['items']:
            comment = item['snippet']['topLevelComment']['snippet']
            comments_20.append([video_id, comment['textDisplay'], comment['authorDisplayName'], comment['publishedAt'], comment['likeCount']])

        if 'nextPageToken' in response:
            response = api_obj.commentThreads().list(part='snippet', videoId=video_id, pageToken=response['nextPageToken'], maxResults=100).execute()
        else:
            break

df3 = pd.DataFrame(comments_20, columns=['id', 'comment', 'author', 'comment_date', 'num_likes'])

df_22 = df_22.reset_index()
df_21 = df_21.reset_index()
df_20 = df_20.reset_index()

sampro_22 = pd.merge(df_22, df1, on='id', how='outer')
sampro_21 = pd.merge(df_21, df2, on='id', how='outer')
sampro_20 = pd.merge(df_20, df3, on='id', how='outer')

sampro_22.to_csv('./sampro/sampro_22.csv', index=False)
sampro_21.to_csv('./sampro/sampro_21.csv', index=False)
sampro_20.to_csv('./sampro/sampro_20.csv', index=False)

sampro_2year = pd.concat([sampro_22, sampro_21, sampro_20])
sampro_2year.head()

In [ ]:
sampro_2year.head()

,video_date,video_title,id,comment,author,comment_date,num_likes,date_diff,market_hour
0,2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk,안녕하세요! 삼프로TV입니다.\r<br>여러분의 응원과 사랑으로 삼프로TV가 구독자...,삼프로TV_경제의신과함께,2022-06-30 13:54:12+00:00,0,-1 days +23:42:07,True
1,2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk,탁프로님 분석 명확하고 구체적이라서 도움 많이 받습니다,상상해보자,2022-07-07 02:52:59+00:00,0,6 days 12:40:54,False
2,2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk,김탁님 분석 최고.,자전거55,2022-07-04 08:00:00+00:00,0,3 days 17:47:55,False
3,2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk,산업구조가 일본에서 한국 그 다음 중국 쪽으로 흘러가는 것. 한국이 살 길은 기술...,꿈이야!,2022-06-30 23:18:01+00:00,0,0 days 09:05:56,False
4,2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk,"누리호 얘기를 듣는데, 사회자분들의 코멘트가 너무 방해된다는 생각이 들정도로 매끄럽...",롱타이,2022-06-30 22:33:27+00:00,1,0 days 08:21:22,False


#### 2.1.2 종목토론실 댓글 크롤링
- 종목토론실 댓글에서 '날짜', '댓글제목', '조회수', '좋아요', 싫어요' 를 크롤링 했습니다.
- 기업대상은 KODEX30의 기업입니다.

In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import datetime as dt


import warnings
warnings.filterwarnings('ignore')

In [ ]:
def get_code(symbol):
    krx = pd.read_csv('./krx_code.csv',encoding='utf-8')
    krx = krx.set_index('한글 종목약명')
    try:
        code = krx.at[symbol,'단축코드']
        return code
    except:
        print('종목명을 다시 확인해주세요.')
        return 0

def get_comment_csv(symbol,page,year,month,day):   
    code = get_code(symbol)
    date_list = [] # 날짜
    comment_list = [] # 댓글
    view_list = [] # 조회수
    good_list = [] # 좋아요
    bad_list = [] # 싫어요
    flag = 0
    for i in range(1,page+1):
        url = f'https://finance.naver.com/item/board.naver?code={code}&page={i}'
        headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36 Edg/100.0.1185.50'}
        res = requests.get(url, headers = headers)
        bs = BeautifulSoup(res.text, 'html.parser')
        for j in range(20):
            try:
                root = bs.find('div',{'class':'section inner_sub'}).find_all('tr',{'onmouseover':'mouseOver(this)'})[j].text.split('\n')
                
                date_list.append(root[1].replace('.','-'))
                
                if len(root) == 14: # 답글
                    comment_list.append('답글:'+root[4])
                    view_list.append(root[10])
                    good_list.append(root[11])
                    bad_list.append(root[12])          
                elif len(root) == 13: # 기본
                    comment_list.append(root[3])
                    view_list.append(root[9])
                    good_list.append(root[10])
                    bad_list.append(root[11])
                else: # 에러
                    comment_list.append('error')
                    view_list.append(0)
                    good_list.append(0)
                    bad_list.append(0)   
            except:
                break
#                 date_list.append('error')
#                 comment_list.append('error')
#                 view_list.append(0)
#                 good_list.append(0)
#                 bad_list.append(0)  
            tp = [int(j) for j in root[1].split()[0].split('.')]
            if dt.datetime(tp[0],tp[1],tp[2]) < dt.datetime(year,month,day):
                flag = 1
                break
        if flag == 1:
            break
        print(f'\r{i}페이지 크롤링 완료.',end='')
        
#         for i in date_list:
#             tp = [int(j) for j in i.split()[0].split('-')]
#             if dt.datetime(tp[0],tp[1],tp[2]) < dt.datetime(year,month,day):
#                 flag = 1
#                 break
#         if flag == 1:
#             break
        
    df = pd.DataFrame()
    df['날짜'] = date_list
    df['댓글'] = comment_list
    df['조회수'] = view_list
    df['좋아요'] = good_list
    df['싫어요'] = bad_list
    return df

- ktop30 리스트 입니다.

In [ ]:
pd.read_excel('./KODEX_KTOP_30_20220629.xlsx',header=2).drop(0,axis=0)

In [ ]:
ktop30_company = pd.read_excel('./KODEX_KTOP_30_20220629.xlsx',header=2).drop(0,axis=0)['종목명']

- 샘플 실행결과 입니다. 
- 인풋 파라미터는 기업이름,최대 페이지수,년,월,일 입니다. 설정한 년/월/일 이후의 댓글이 나오면 크롤링을 멈추게 됩니다. 

In [ ]:
df = get_comment_csv('미래에셋증권',500,2022,6,1) # sample
df

In [ ]:
# 실행 건너뛰기
for i in range(3):
    df = get_comment_csv(list(ktop30_company)[i],100000,2020,6,1)
    #df = df[df['날짜']=='2022-07-01']
    df.to_csv(f'./src/2year_time/{list(ktop30_company)[i]}_2year.csv')

In [ ]:
# 크롤링된 데이터 확인
pd.read_csv('./src/year_time/삼성전자_year.csv')

### 2.2 비정형 데이터 전처리

#### 2.2.1 댓글 전처리
- 온전한 한글문자 이외에 모든 부분을 제거했습니다.
- 초성,알파벳,특수문자,이모티콘 등이 제거 되었습니다.


- 예시 코드입니다.

In [ ]:
import pandas as pd

In [ ]:
target = pd.read_csv('./src/2year_time/삼성화재_2year.csv')
target

In [ ]:
def clean_sents_df(company):
    try:
        target = pd.read_csv(f'./src/{company}_2year.csv',encoding='utf8').drop('Unnamed: 0',axis=1)
    except:
        target = pd.read_csv(f'./src/{company}_2year.csv',encoding='utf8')

    if company == 'sampro':
        target.rename(columns={'comment':'댓글'},inplace=True)
    df = target
    df['정제된 댓글'] = df['댓글'].str.replace('\\[삭제된 게시물의 답글\\]',' ')
    df['정제된 댓글'] = df['정제된 댓글'].str.replace('답글:',' ')
    df['정제된 댓글'] = df['정제된 댓글'].str.replace('[^가-힣]',' ').str.replace(' +',' ').str.strip()
    df = df[df['정제된 댓글'] != '']
    df = df.reset_index(drop=True)
    return  df

#### 2.2.2 댓글 토크나이징

- Bert 모델링을 위한 labeling 사전작업 입니다.

In [ ]:
def corpus_save(company):
    df = clean_sents_df(company)
    df['정제된 댓글 길이'] = [len(str(i)) for i in df['정제된 댓글']]
    df = df[df['정제된 댓글 길이'] > 5]

    tp = [str(i) for i in list(df['정제된 댓글'])]
    save = '\n'.join(tp)
    f = open("./corpus_target.txt", 'a',encoding='utf8')
    f.write(save)
    f.close()
    
def corpus_init():
    ktop30_company = pd.read_excel('./src/KODEX_KTOP_30_20220629.xlsx',header=2).drop(0,axis=0)['종목명']
    company_set = list(ktop30_company)
    company_set.append('sampro')
    f = open("./corpus_target.txt", 'w',encoding='utf8')
    f.write('')
    f.close()
    for company in company_set:
        corpus_save(company)

def return_tokenizer():
    corpus = DoublespaceLineCorpus("./src/corpus_target.txt",iter_sent=True)
    noun_extractor = LRNounExtractor_v2(verbose=True)
    nouns = noun_extractor.train_extract(corpus)
    scores = {word:score.score for word, score in nouns.items()}
    tokenizer = LTokenizer(scores=scores)
    return tokenizer

In [ ]:
corpus_init()
tokenizer = return_tokenizer()

- 3181878 개의 네이버 종목토론방, 유튜브 댓글을 학습시키고, 학습된 tokenizer를 이용하여 전처리 댓글을 토큰화 시킵니다.

In [ ]:
target_df = clean_sents_df('삼성전자')
target_df['토큰화 댓글'] = [tokenizer(str(i)) for i in target_df['정제된 댓글']]
target_df

#### 2.2.3 댓글 레이블링
- fear_words_set 과 greed_words_set을 설정하고 토크나이징댓글의 요소가 fear 단어집에 있으면 -1씩 greed 단어집에 있으면 +1씩 부여했습니다.
- 양수인경우 greed, 음수인경우 fear로 labeling 했습니다. 0 인경우 일단 데이터프레임에 기록되고, 최종적으로 train_data 선정시에는 제외됩니다.

In [ ]:
def labeling(target_df):
    
    f = open("./neg_pol_word.txt", 'r',encoding='utf8')
    words = f.readlines()
    f.close()
    fear_words_set = {word.strip('\n') for word in words}

    f = open("./pos_pol_word.txt", 'r',encoding='utf8')
    words = f.readlines()
    f.close()
    greed_words_set = {word.strip('\n') for word in words}
    
    label_score = []
    for token_list in target_df['토큰화 댓글']:
        sent_score = 0
        for token in token_list:
            if token in fear_words_set:
                sent_score -= 1
            elif token in greed_words_set:
                sent_score += 1

        if sent_score < 0:
            label_score.append(-1)
        elif sent_score > 0:
            label_score.append(1)
        else:
            label_score.append(0)
            
    target_df['label'] = label_score
    
    return target_df

#### 2.2.4 학습데이터 생성

- 학습데이터는 종목토론실 댓글에서 3만개, 유튜브 댓글에서 1만개 랜덤추출 했습니다.

In [ ]:
def setting_train_data():
    company = list(pd.read_excel('./KODEX_KTOP_30_20220715.xls',header=2)['종목명'][1:])
    dominance = list(pd.read_excel('./KODEX_KTOP_30_20220715.xls',header=2)['비중(%)'][1:])

    train_data = pd.DataFrame(columns=['날짜','댓글','조회수','좋아요','싫어요','정제된 댓글','토큰화 댓글','label'])
    for idx in range(30):
        target_df = clean_sents_df(company[idx])
        target_df['토큰화 댓글'] = [tokenizer(str(i)) for i in target_df['정제된 댓글']]
        
        label_df = labeling(target_df)
        
        label_df = label_df[label_df['label'] != 0] # except label:0 
        label_df = label_df.sample(int(30000*dominance[idx])+1,replace=True) # replace option : 샘플수가 부족할경우 중복이 생김 ** 추후 수정
        train_data = train_data.append(label_df)
    
    return train_data

In [ ]:
train_data = setting_train_data()
train_data.reset_index(inplace=True)
train_data.drop('index',axis=1,inplace=True)
train_data

In [ ]:
# 삼프로
target_df = clean_sents_df('sampro')
target_df['토큰화 댓글'] = [tokenizer(str(i)) for i in target_df['정제된 댓글']]
label_df = labeling(target_df)

label_df = label_df[label_df['label'] != 0] # except label:0 
label_df = label_df.sample(10000,replace=True) # replace option : 샘플수가 부족할경우 중복이 생김 ** 추후 수정
train_data = train_data.append(label_df)

train_data = train_data.loc[:,['정제된 댓글','label']].reset_index().drop('index',axis=1)

train_data

#### 2.2.5 날짜별 데이터 정리
+ krx30 종목토론방 댓글, 유튜브 댓글을 각 날짜별로 정리한 후, 2022-06-01 ~ 2022-06-30 한 달의 데이터를 예측에 사용했습니다.

In [ ]:
import pandas as pd

# 파일 이름 불러오기
ktop30_company = pd.read_excel('../junho/KODEX_KTOP_30_20220629.xlsx',header=2).drop(0,axis=0)['종목명']
company_set = list(ktop30_company)
company_set.append('sampro1')
company_set.append('sampro2')

# 모든 데이터를 하나의 리스트에 저장
folder_path = "../sh/data/preprocessed"
alldf = []
for company in company_set:
    df = pd.read_csv(folder_path + "/" + company + ".csv")
    if "sampro" in company:
        df2 = df[["video_date","댓글","정제된 댓글"]]
        df2.columns = ["날짜","댓글","정제된 댓글"]
    else:
        df2 = df[["날짜","댓글","정제된 댓글"]]
    alldf.append(df2)

# 날짜 형식을 년-월-일 로 
for df in alldf:
    date_list = []
    print(len(df["날짜"]))
    for i in range(len(df["날짜"])):
        date_list.append(df["날짜"][i][:10])
    df["날짜"] = date_list

952452
72156
60021
131160
213204
26984
23062
52409
38012
126374
6170
25111
180665
604376
36393
22913
31351
31095
17665
128774
32078
120000
112781
78324
35666
27183
26421
38197
125273
17057
187654
270612


In [ ]:
alldf[0]

,날짜,댓글,정제된 댓글
0,2022-07-11,요즘 한전 게시판 조용하네!ㅋ,요즘 한전 게시판 조용하네
1,2022-07-11,1% 올려야 함,올려야 함
2,2022-07-11,=자신들잘못된죄들=모르는=내로남불들=척결...,자신들잘못된죄들 모르는 내로남불들 척결
3,2022-07-11,난 내일 가요,난 내일 가요
4,2022-07-11,실적으로 이만한 실적내는 기업 있으면 나...,실적으로 이만한 실적내는 기업 있으면 나
...,...,...,...
952447,2020-06-01,내8자,내 자
952448,2020-06-01,하락낙폭저지 의 길,하락낙폭저지 의 길
952449,2020-06-01,주주님들 오랜만에 오르니까 좋네요 ㅎㅎㅎ,주주님들 오랜만에 오르니까 좋네요
952450,2020-06-01,코덱스200선물인버스 곱배기..,코덱스 선물인버스 곱배기


In [ ]:
# 2022-06-01 ~ 2022-06-30 날짜 데이터를 각각 하나로 뭉침
target_date = ['2022-06-01', '2022-06-02', '2022-06-03', '2022-06-04', '2022-06-05',
               '2022-06-06', '2022-06-07', '2022-06-08', '2022-06-09', '2022-06-10',
               '2022-06-11', '2022-06-12', '2022-06-13', '2022-06-14', '2022-06-15',
               '2022-06-16', '2022-06-17', '2022-06-18', '2022-06-19', '2022-06-20',
               '2022-06-21', '2022-06-22', '2022-06-23', '2022-06-24', '2022-06-25',
               '2022-06-26', '2022-06-27', '2022-06-28', '2022-06-29', '2022-06-30',
               '2022-05-31']
td = sorted(target_date,reverse=True)
all_date = []
for df in alldf:
    date_idx = dict()
    for i in range(len(df["날짜"])):
        if df["날짜"][i] in date_idx:
            continue
        else:
            if df["날짜"][i] in target_date:
                date_idx[df["날짜"][i]] = i
    all_date.append(date_idx)
for j in all_date:
    for i in range(len(td)):
        if td[i] in j:
            continue
        else:
            j[td[i]] = 0
for i in all_date:
    if i["2022-05-31"] == 0:
        continue
    for j in range(len(td)-2,0,-1):
        if i[td[j]] == 0:
            i[td[j]] = i[td[j+1]]
for i in range(30):
    for c in range(32):
        df = alldf[c]
        date_idx = all_date[c]
        if c == 0:
            df_day = df.iloc[date_idx[td[i]]:date_idx[td[i+1]]]
        elif c == 30:
            continue
        else:
            df_today = df.iloc[date_idx[td[i]]:date_idx[td[i+1]]]
            df_day = pd.concat([df_day,df_today])
    df_day.to_csv(td[i] + ".csv")

df_day

,날짜,댓글,정제된 댓글
124479,2022-06-01,내일도좃되나재룡좃,내일도좃되나재룡좃
124480,2022-06-01,믿고 가는 애플,믿고 가는 애플
124481,2022-06-01,삼전 목요일 100% 상승!,삼전 목요일 상승
124482,2022-06-01,홍어들 잡아오면ㆍ백만원 드립니다ㆍ,홍어들 잡아오면 백만원 드립니다
124483,2022-06-02,답글:홍어들 잡아오면ㆍ백만원 드립니다ㆍ,홍어들 잡아오면 백만원 드립니다
...,...,...,...
15215,2022-06-01,왁굳형사랑해,왁굳형사랑해
15216,2022-06-01,아니...우왁굳이랑 이세돌은 어디까지 가는거야...ㅋㅋㅋㅋ,아니 우왁굳이랑 이세돌은 어디까지 가는거야
15217,2022-06-01,미군의 IVAS나 로블록스처럼 미래에는 게임을 넘어서 다양한 융복합 시대가 오겠지만...,미군의 나 로블록스처럼 미래에는 게임을 넘어서 다양한 융복합 시대가 오겠지만 투자를...
15218,2022-06-01,상균쿤..?,상균쿤


## chapter 3. koBERT 모델링
+ koBERT model
    + BERT 모델이란 구글에서 개발한 사전학습 언어모델입니다.<br/>
    한국어는 복잡한 언어의 특성 때문에 BERT 모델의 예측률이 상대적으로 떨어지는 편입니다.<br/>
    koBERT는 이 문제를 해결하기 위해 만들어진 한국어 버전 BERT모델이라 할 수 있습니다.
    + 이 모델에서는 koBERT 모델을 사용하여 4만개의 fear/greed labeled data를 학습하였으며,<br/>
    이후 30일간의 약 20만개정도의 댓글의 fear/greed 점수를 측정하였습니다.

### 3.1 모델 학습
+ 2.2.4에서 생성된 학습 데이터를 모델 학습에 사용하였습니다.

In [4]:
# 패키지 설치
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch
!pip install gluonnlp
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49.1 MB 2.4 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 344 kB 33.3 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595740 sha256=d7451cf7e6770bfcfe9f7a45779e5bf2740c66dd0bb4f2dff54bc79263505792
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 19.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.p

In [6]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [7]:
# google colab 패키지 사용
from google.colab import drive

# 구글 드라이브 연결
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import pandas as pd

# GPU 설정
device = torch.device("cuda:0")

# 데이터 적재
file_path = "/content/drive/MyDrive/CNN/댓글/train_data.csv"
df = pd.read_csv(file_path)

df

,Unnamed: 0,정제된 댓글,label
0,0,개미가 털고 나가야 오른다,1
1,1,공포에 사고 환희에 팔아라,0
2,2,만원을 위하여 다음주 실적 발표하,0
3,3,역시 개미가 나가야 오른다,1
4,4,낼 실적발표,0
...,...,...,...
40010,40010,엄청난 열기와 성원에 힘입어 진행된 하반기 투자전략 세미나 지금 온라인으로 한 달간...,1
40011,40011,강의 신청하기 투자의 정답을 찾는 책읽기 시즌 사경인의 실전투자 아카데미 리마스터 ...,1
40012,40012,자난 월은 월을 이겨낸 분들의 날이였죠 지금도 공포에 매수 한 분들의 좋은 결과 있...,0
40013,40013,에 개인투자자들이 너무 많이 몰려서 손실을 볼까 염려하는 마음에 걱정해주시는 것은 ...,1


In [10]:
# 댓글과 라벨만 사용, 훈련 데이터와 테스트 데이터 나눔
df_nn = df[["정제된 댓글","label"]]
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df_nn["정제된 댓글"],df_nn["label"],test_size=0.2,random_state=11)
df_train = pd.concat([x_train,y_train],axis = 1)
df_test = pd.concat([x_test,y_test],axis = 1)
print(df_train["label"].unique())
df_train.head()

[0 1]


,정제된 댓글,label
21613,분기 실적만 기다리며 추매중,0
26881,코로나 수혜주 아니였냐 실적도 좋은데,0
25626,실적발표를 빨리한다는건좋은신호로 받아들였,0
5718,허매수가 매도의 배다 절대못오른다,1
21414,누루기 분기 실적,0


In [11]:
# 파라미터 설정 
max_len = 128
batch_size = 64
warmup_ratio = 0.1
num_epochs = 1
max_grad_norm = 1
log_interval = 100
learning_rate =  0.00004

# 데이터셋을 koBERT 모델에 맞게 변형하는 클래스
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i]) for i in dataset[sent_idx]]
        self.labels = [np.int32(i) for i in dataset[label_idx]]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))  


# koBERT 모듈을 적재하여 모델을 학습시키는 분류기 
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)


In [12]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [13]:
# 토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [14]:
# 훈련데이터, 테스트데이터 전처리
data_train = BERTDataset(df_train, "정제된 댓글", "label", tok, max_len, True, False)
data_test = BERTDataset(df_test, "정제된 댓글", "label", tok, max_len, True, False)
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [15]:
# 모델 생성
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [16]:
# optimizer 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [17]:
# 모델 학습
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

train_dataloader

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step() 
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  del sys.path[0]


  0%|          | 0/501 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 0.73428875207901 train acc 0.453125
epoch 1 batch id 101 loss 0.0022356719709932804 train acc 0.875309405940594
epoch 1 batch id 201 loss 0.0009426305186934769 train acc 0.9367226368159204
epoch 1 batch id 301 loss 0.000408878899179399 train acc 0.9576931063122923
epoch 1 batch id 401 loss 0.0003314330824650824 train acc 0.968165523690773
epoch 1 batch id 501 loss 0.00044410067494027317 train acc 0.9745197105788423
epoch 1 train acc 0.9745197105788423


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/126 [00:00<?, ?it/s]

epoch 1 test acc 0.9997519841269841


In [18]:
# 모델 저장
torch.save(model, '/content/drive/MyDrive/CNN/KoBERT_model_dr_0.5.pt') 
torch.save(model.state_dict(), '/content/drive/MyDrive/CNN/model_state_dict_dr_0.5.pt')

### 3.2 데이터 예측
+ 2.2.5에서 생성된 2022-06-01 ~ 2022-06-30 댓글 데이터를 저장된 모델을 사용하여 fear/greed 점수를 예측하였습니다.
+ 각각의 댓글은 fear/greed로 예측됩니다.<br/>
이때, 예측된 fear/greed는 각각 점수를 가지고 있으며, 이 점수로 각 댓글의 fear/greed 점수를 산출합니다.
+ 각 날짜별 모든 댓글의 fear/greed 점수를 평균을 낸 것을 그날의 fear/greed 점수로 합니다.<br/>
이를 0 ~ 100 사이의 값을 가지도록 조정해줍니다. 0에 가까울수록 fear이며, 100에 가까울수록 greed입니다.

In [19]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [20]:
# GPU 설정
device = torch.device("cuda:0")

# 버트모델 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

# 파라미터 설정 
max_len = 128
batch_size = 64
warmup_ratio = 0.1
num_epochs = 1
max_grad_norm = 1
log_interval = 100
learning_rate =  0.00004

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [21]:
# KoBERT에 입력될 데이터셋 정리
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))  

# fear/greed 평가
def new_softmax(a) : 
    c = np.max(a)
    exp_a = np.exp(a-c) 
    sum_exp_a = np.sum(exp_a)
    y = (exp_a / sum_exp_a) * 100
    return np.round(y, 3)

# 분류기 생성
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

# 예측 모델 설정
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)

        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
            min_v = min(logits)
            total = 0
            probability = []
            logits = np.round(new_softmax(logits), 3).tolist()
            for logit in logits:
                print(logit)
                probability.append(np.round(logit, 3))

            if np.argmax(logits) == 0: emotion = "fear"
            elif np.argmax(logits) == 1: emotion = 'greed'

            probability.append(emotion)
            print(probability)
    return probability

In [22]:
## 학습 모델 로드
model = torch.load('/content/drive/MyDrive/CNN/KoBERT_model_dr_0.5.pt')
model.load_state_dict(torch.load('/content/drive/MyDrive/CNN/model_state_dict_dr_0.5.pt'))

<All keys matched successfully>

In [23]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


+ 30일치 데이터를 모두 돌릴 경우 메모리가 부족하여 1일치씩 예측을 진행

In [26]:
# 데이터 불러오기
target_date = [0,
               '2022-06-01', '2022-06-02', '2022-06-03', '2022-06-04', '2022-06-05',
               '2022-06-06', '2022-06-07', '2022-06-08', '2022-06-09', '2022-06-10',
               '2022-06-11', '2022-06-12', '2022-06-13', '2022-06-14', '2022-06-15',
               '2022-06-16', '2022-06-17', '2022-06-18', '2022-06-19', '2022-06-20',
               '2022-06-21', '2022-06-22', '2022-06-23', '2022-06-24', '2022-06-25',
               '2022-06-26', '2022-06-27', '2022-06-28', '2022-06-29', '2022-06-30']

# for date in target_date:
#     file_path = target_date[date] + ".csv"
#     df = pd.read_csv(file_path)

#     # 데이터 예측 실행
#     df_pred = []
#     for comment in df["정제된 댓글"]:
#         fg_result = predict(comment)
#         score = abs(fg_result[0] - fg_result[1])
#         if fg_result[2] == "fear":
#             score = -score
#         df_pred.append(score)

#     # 예측 결과를 csv 파일로 저장
#     df["score"] = df_pred
#     file_path_2 = target_date[date] + "_score.csv"
#     df.to_csv(file_path_2)

df = pd.read_csv("/content/drive/MyDrive/CNN/date/2022-06-01.csv")

# 데이터 예측 실행
df_pred = []
for comment in df["정제된 댓글"]:
    fg_result = predict(comment)
    score = abs(fg_result[0] - fg_result[1])
    if fg_result[2] == "fear":
        score = -score
    df_pred.append(score)

# 예측 결과를 csv 파일로 저장
df["score"] = df_pred
file_path_2 = "/content/drive/MyDrive/CNN/score/2022-06-01_score.csv"
df.to_csv(file_path_2)

# 그날의 fear/greed 점수 산출
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/CNN/score/2022-06-01_score.csv")
df = df[["날짜","score"]]
df = df[df["날짜"] == "2022-06-01"]
df_mean = df.groupby("날짜").mean()

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
99.76200103759766
[0.238, 99.762, 'greed']
0.05999999865889549
99.94000244140625
[0.06, 99.94, 'greed']
24.924999237060547
75.07499694824219
[24.925, 75.075, 'greed']
0.23000000417232513
99.7699966430664
[0.23, 99.77, 'greed']
0.08399999886751175
99.91600036621094
[0.084, 99.916, 'greed']
0.05299999937415123
99.9469985961914
[0.053, 99.947, 'greed']
0.23100000619888306
99.76899719238281
[0.231, 99.769, 'greed']
94.54199981689453
5.458000183105469
[94.542, 5.458, 'fear']
0.09099999815225601
99.90899658203125
[0.091, 99.909, 'greed']
99.96199798583984
0.03799999877810478
[99.962, 0.038, 'fear']
99.79399871826172
0.20600000023841858
[99.794, 0.206, 'fear']
23.756000518798828
76.24400329589844
[23.756, 76.244, 'greed']
99.90499877929688
0.0949999988079071
[99.905, 0.095, 'fear']
52.00400161743164
47.99599838256836
[52.004, 47.996, 'fear']
99.9219970703125
0.07800000160932541
[99.922, 0.078, 'fear']
99.7699966430664
0.23000000417232513
[99.77, 0.23, 'fear

In [ ]:
df_mean

## chapter 4. 시각화 및 서비스